In [ ]:
import os
import sys
sys.path.append("./")
import yaml
import pickle
import argparse

import numpy as np
import pandas as pd
import networkx as nx

from data.process import LoadDataset
from utils.graph_utils import loadGraph, merge_graph_attributes, get_node_att_value, map_att_to_node
from utils.functions import LoadConfig

from data.Augmentation import mutation_anchored_subgraphs

In [ ]:
edgepath = 


In [ ]:
def NoteBookParse():
    parser = argparse.ArgumentParser(description='Run DGI model')
    parser.add_argument('--config_path', type=str, default=os.path.join(os.getcwd(), '../config/run.yaml'), help='Path to the run configuration file')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size')
    parser.add_argument('--num_workers', type=int, default=4, help='Number of workers for data loading')
    parser.add_argument('--wandb_key', type=str, default=None, help='Wandb API key')
    parser.add_argument('--entity_name', type=str, default='shmoon', help='Wandb entity name')
    parser.add_argument('--project_name', type=str, default='DeepResidueCluster', help='Wandb project name')
    parser.add_argument('--wandb_run_name', type=str, default='DGI', help='Wandb run name')
    parser.add_argument('--wandb_run_id', type=str, default=None, help='Wandb run id')
    parser.add_argument('--load_pretrained', action='store_true', help='Load pretrained model')
    parser.add_argument('--nowandb', action='store_true', help='Do not use wandb')
    
    args, unknown = parser.parse_known_args() 
    return args

In [ ]:
ConfigPATH = '../config/run.yaml'
with open(ConfigPATH, 'r') as f:
    model_config = yaml.safe_load(f)
config = argparse.Namespace(**model_config)

In [ ]:
G = loadGraph(config.Graph_PATH)
print(G)
nodes_in_G = list(G.nodes())

In [ ]:
feat_df = pd.read_csv(config.Feature_PATH)

# Connection (Edge) Analysis

<!-- ## Original Edge Table -->

In [ ]:
# original_edge_df = pd.read_csv(edgepath)

In [ ]:
# all_semicol_node_df = original_edge_df[(original_edge_df['uniprot1'].str.contains(';')) | (original_edge_df['uniprot2'].str.contains(';'))][['uniprot1', 'uniprot2', 'pdb_code', 'chain_flag']]
# both_semicol_node_df = original_edge_df[(original_edge_df['uniprot1'].str.contains(';')) & (original_edge_df['uniprot2'].str.contains(';'))][['uniprot1', 'uniprot2', 'pdb_code', 'chain_flag']]

In [ ]:
# all_semicol_node_df.head(2)

In [ ]:
# both_semicol_node_df['edge_key'] = list(zip(
#     np.minimum(both_semicol_node_df['uniprot1'], both_semicol_node_df['uniprot2']),
#     np.maximum(both_semicol_node_df['uniprot1'], both_semicol_node_df['uniprot2'])
# ))

# all_semicol_node_df['edge_key'] = list(zip(
#     np.minimum(all_semicol_node_df['uniprot1'], all_semicol_node_df['uniprot2']),
#     np.maximum(all_semicol_node_df['uniprot1'], all_semicol_node_df['uniprot2'])
# ))

In [ ]:
# all_semicol_node_df.reset_index(drop=False, inplace=True)
# both_semicol_node_df.reset_index(drop=False, inplace=True)


In [ ]:
# unique_index = all_semicol_node_df.index.difference(both_semicol_node_df.index)
# filtered_df = all_semicol_node_df.loc[unique_index]
# len(filtered_df)

In [ ]:
# both_semicol_node_df.pdb_code.value_counts()

In [ ]:
# all_semicol_node_df.chain_flag.value_counts()

In [ ]:
# both_semicol_node_df.chain_flag.value_counts()

<!-- ## Graph Connection -->

In [ ]:
graph_connections = nx.to_pandas_edgelist(G)
graph_connections

In [ ]:
graph_connections.source = graph_connections.source.apply(lambda x: x.split('_')[0])
graph_connections.target = graph_connections.target.apply(lambda x: x.split('_')[0])

g_node1 = np.minimum(graph_connections['source'], graph_connections['target'])
g_node2 = np.maximum(graph_connections['source'], graph_connections['target'])
graph_connections['edge_key'] = list(zip(g_node1, g_node2))
graph_connections.drop_duplicates(subset=['edge_key'], inplace=True)
only_semicol_in_graph = pd.concat([graph_connections[graph_connections.target.str.contains(';')], graph_connections[graph_connections.source.str.contains(';')]])

In [ ]:
only_semicol_in_graph

In [ ]:
merged_df = pd.merge(
    only_semicol_in_graph,
    all_semicol_node_df[['edge_key', 'pdb_code', 'chain_flag']],
    on='edge_key',
    how='left'
)
merged_df.drop_duplicates(subset=['edge_key'], inplace=True)

In [ ]:
merged_df.at[merged_df.index[0], 'pdb_code'] = '5z23'
merged_df.at[merged_df.index[0], 'chain_flag'] = 'inter-chain'

In [ ]:
merged_df

<!-- ## Remove Unrelated Nodes -->

In [ ]:
# removed_nodes = ['q92522;p10412', 'p06899;p04908', 'q92522;p10412', 'p49450;p68431', 'p49450;p68431-1']

In [ ]:
# newGraphPATH =
# nodes_to_delete = [
#     n for n in G.nodes() 
#     if any(pattern in n for pattern in removed_nodes)
# ]

# G.remove_nodes_from(nodes_to_delete)

# with open(newGraphPATH, 'wb') as f:
#     pickle.dump(G, f)

In [ ]:
# print(G)

In [ ]:
# 

In [ ]:
# 

# Graph Analysis

In [ ]:
CCs = [G.subgraph(c).copy() for c in nx.connected_components(G)]

In [ ]:
prot

In [ ]:
single_cc = []
non_mut_cc = []
non_mut_onlycopy_cc = []
non_mut_noncopy_cc = []
has_mut_cc = []

for i in range(len(CCs)):
    cc = CCs[i]
    df_cc = nx.to_pandas_edgelist(cc)
    df_cc['uniprot1'] = df_cc['source'].apply(lambda x: x.split('_')[0])
    df_cc['uniprot2'] = df_cc['target'].apply(lambda x: x.split('_')[0])
    prot = set(df_cc['uniprot1'].unique()).union(df_cc['uniprot2'].unique())
    
    if len(prot) == 1:
        print(prot)
        single_cc.extend(list(prot))
        
        attG = map_att_to_node(cc, feat_df, use_cols=['from_copy', 'is_mut'])
        cnt_mut = get_node_att_value(attG, 'is_mut')
        sum_mut = sum(cnt_mut)
        if sum_mut == 0:
            non_mut_cc.append(i)
            cnt_copy = get_node_att_value(attG, 'from_copy')
            sum_copy = sum(cnt_copy)
            if sum_copy == attG.number_of_nodes():
                non_mut_onlycopy_cc.append(i)
            elif sum_copy == 0 :
                non_mut_noncopy_cc.append(i)
        else:
            has_mut_cc.append(i)

In [ ]:
print("Total Number of Chain", len(CCs))
print("-----------------")
print("Single Protein CCs", len(single_cc), "(", round((len(single_cc)/len(CCs))*100, 2),'%)')
print("-----------------")
print("Mut CCs among one-chain only", len(has_mut_cc), "(", round((len(has_mut_cc)/len(CCs))*100, 2),'%)')
print("Non-Mut CCs among one-chain only", len(non_mut_cc), "(", round((len(non_mut_cc)/len(CCs))*100, 2),'%)')
print("-----------------")
print("Non-Mut only NonCopy CCs", len(non_mut_noncopy_cc), "(", round((len(non_mut_noncopy_cc)/len(CCs))*100, 2),'%)')
print("Non-Mut Only Copy CCs", len(non_mut_onlycopy_cc), "(", round((len(non_mut_onlycopy_cc)/len(CCs))*100, 2),'%)')

# Node Features from Table

In [ ]:
df1 = # 
df2 = # 

In [ ]:
df1.info()

In [ ]:
df1.copyindex.fillna(0, inplace=True)
nodes_in_df = df1.node_id.values
target_set = set(nodes_in_df)

In [ ]:
skip_nodes = [n for n in nodes_in_G if n not in target_set]
skip_nodes

In [ ]:
df3 = 

In [ ]:
df3[df3.node_id.str.contains(';')].uniprot_id.unique()

In [ ]:
df3[df3.node_id.str.contains(';')]